In [1]:
from pathlib import Path
import shutil

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import locale
#from html.parser import HTMLParser
# import website_func.py to use its functions
from website_func import *
import os #To read the file
#reload every module each time
%load_ext autoreload
%autoreload 2

# Some quick analysis of the data
We received a data set of .html file containing the website content of recipes.
We wanted to sort them by website, in order to, more easily, find a pattern among them. This will enable us to do the "scraping" of the pages. First we thought about moving the files in a folder corresponding to their website, but it would be a waste of time and a big computational effort. Thus, we came up with a (probably) faster solution : we could simply write the name of the file within its corresponding website folder. By inspecting the files, we saw that the first line was always containing a comment with the name of the file and the complete website. Using readlines and split, we could easily retrieve the name of the website.

We launched this process, but an alarm appeared describing a Trojan virus in the file "1c2cb6f0df04cf5a9d0baa116c6aa7bb.html". 
We had then to quarantine or mabye remove the file, as we have quite enough
By doing so, we remarked the file "msg.log" that could help us into fastering the processus as its content is formed of the name of the file together with its website. [SHOW] We obserbed that occasionally a line containing other info that the content we want can appear. We need to ignore this
Also we noticed there are no file extensions other than .html and .log. 

In [14]:
retrieve_website_from_log()

FileNotFoundError: [Errno 2] No such file or directory: 'recipePages.zip\\recipePages\\msg.log'

In [4]:
# We can find in the folder that, excepted the html files, there is only the log

# Get all filenames (i.e. path) that are in recipePages folder 
pathlist = Path("recipePages/").glob('**/*')
i = 0
for path in pathlist:
     # because path is object not string
    path_in_str = str(path)
    if not path_in_str.endswith(".html"):
        print(path_in_str)

recipePages\msg.log


In [5]:
website_count=pd.Series()

for (root,dirs,files) in os.walk('SortedFiles', topdown="True"):   
        for website in dirs:
            
            filename = "SortedFiles/" + website +"/filesName.txt"
            
            f = open(filename, "r")
            number_of_link = len(f.readlines())
            line_to_add=pd.Series(number_of_link,index=[website])
            website_count = website_count.append(line_to_add)

FileNotFoundError: [Errno 2] No such file or directory: 'SortedFiles/.ipynb_checkpoints/filesName.txt'

In [6]:
website_count.sort_values(ascending=False,inplace=True)

In [7]:
website_count_used=website_count[website_count.values>1000]

In [8]:
website_list_used=website_count_used.index.tolist()

In [13]:
#Words that have to be removed from the string in the website to keep only the ingredient:
list_ingredient_to_remove = ["teaspoon" , "cup", "cups","teaspoons","ounce","ounces","pounds","pound","tablespoons","chopped","quarts","fresh",\
                            "light","plain","popped","medium","shredded","bunch","tablespoon","to","taste","pinch","freshly","ground",\
                            "canned","carcass","cubes","dried","frozen","all","purpose","grated","minced","degree","degrees","optional",\
                            "jar","quartered","marinated","strips","strip","asian","stalks","package","can","cans","box","container",\
                            "evaporated","bottle","cans","extract","squares","german","slices","crushed","uncooked","seasoning","small",\
                            "sweet","packed","sliced","heavy","condensed","finely","long","grain","sweetened","firmly","cooked","crusts","lean",\
                            "caps","mix","steaks","large","instant","crumbs","semisweet","distilled","packages","pint","miniature","preserves",\
                            "processed","chunky","prepared","seasoned","american","stock","top","sirloin","tip","peeled","spears","leaves",\
                            "belgian","bob","italian","bottles","boneless","roast","breast","meat","tenderloins","granules","chips","converted",\
                            "containers","coleslaw","florets","cube","cubed","diced","crumbled","head","thighs","halves","drumsticks","blend",\
                            "leaf","bow","tie","stuffing","dry","boiling","cover","coarse","kosher","extra","virgin","fat","free","fluid","hot",\
                            "chops","loin","shoulder","roasted","for","frying","diced","thawed","bone","hocks","shucked","leftover","corned",\
                            "weed","summer","wild","whole","very","ripe","unbaked","crust","unsalted","unsifted","unsweetened","drops","drop",\
                            "paste","thickly","swiss","skinless","flavored","quart","puff","kernels","kernel","cracked","as","tortillas","chunk",\
                            "dash","dark","hash","brown","hashbrown","hungarian","rinsed","thin","thinly","thick","stewed","cleaned","gelatin",\
                            "sifted","skim","slivered","sprigs","sage","roma","romano","pasta","reduced","sodium","refried","refrigerated",\
                            "portobello","nuggets","tater","pinto","pudding","pitted","pinches","relish","penne"]

list_ingredient_to_remove = sorted(list_ingredient_to_remove)
print(list_ingredient_to_remove)



['all', 'american', 'as', 'asian', 'belgian', 'blend', 'bob', 'boiling', 'bone', 'boneless', 'bottle', 'bottles', 'bow', 'box', 'breast', 'brown', 'bunch', 'can', 'canned', 'cans', 'cans', 'caps', 'carcass', 'chips', 'chopped', 'chops', 'chunk', 'chunky', 'cleaned', 'coarse', 'coleslaw', 'condensed', 'container', 'containers', 'converted', 'cooked', 'corned', 'cover', 'cracked', 'crumbled', 'crumbs', 'crushed', 'crust', 'crusts', 'cube', 'cubed', 'cubes', 'cup', 'cups', 'dark', 'dash', 'degree', 'degrees', 'diced', 'diced', 'distilled', 'dried', 'drop', 'drops', 'drumsticks', 'dry', 'evaporated', 'extra', 'extract', 'fat', 'finely', 'firmly', 'flavored', 'florets', 'fluid', 'for', 'free', 'fresh', 'freshly', 'frozen', 'frying', 'gelatin', 'german', 'grain', 'granules', 'grated', 'ground', 'halves', 'hash', 'hashbrown', 'head', 'heavy', 'hocks', 'hot', 'hungarian', 'instant', 'italian', 'jar', 'kernel', 'kernels', 'kosher', 'large', 'leaf', 'lean', 'leaves', 'leftover', 'light', 'loin',

In [14]:
#Create the data from the information found on the HTML pages

recipe_data = pd.DataFrame(columns = ['Website','Recipe', 'Ranking', 'Reviews', 'Ingredients'])  
list_unique_ingredients = []

for (root,dirs,files) in os.walk('recipePagesT', topdown="True"):  
    for webpage in files:
        filename= 'recipePagesT/'+webpage
        
        #Try to open it, and if it doesn't, we don't analyse this webpage
        try:
            f = open(filename,'r')
            first_line=f.readline()
            f.close()
        except:
            print("We can't read the page:",filename)
            continue

        website = str(first_line).split("/")[2].strip("www.")
        
        #Determine if we have to spare the data
        if website in website_list_used :
            
            #Read the data from the first website (allrecipes.com)
            if website in website_list_used[0]:
                #print(filename)
                try:
                    soup = BeautifulSoup(open(filename), "html.parser")
                except:
                    print("Beautifulsoup can't read the page:",filename)
                    continue
                
                recipe_name = soup.find('span', class_='itemreviewed')
                
                #We only take recipes and not searching pages
                if recipe_name is None:
                    print("We don't care about this page: ",filename)
                    continue
                recipe_name = recipe_name.text
                
                rating_html = soup.find('img', class_='rating')
                
                #Determine position of the ranking in the string
                start_rank = 59
                end_rank = 62
                rating = rating_html['title'][start_rank:end_rank]
                
                #print(recipe_name)
                #print(rating)
                
                #Determine the number of reviews:
                if soup.find('span', class_='count') is not None:
                    review_html = soup.find('span', class_='count').text
                    review = int(review_html.replace(',',''))
                else:
                    review = 0
                #print('Reviews :',review)
                
                #Find the preparation time:
                prepare_time = np.nan
                soup1 = soup.find('span', class_='totalTime')
                if soup1 is not None:
                    
                    prepare_time_html = soup1.find('span', class_='value-title')
                    #Determine position of the time in the string
                    start_prepare_time = 2
                    end_prepare_time = len(prepare_time_html['title']) 
                    prepare_time_not_converted = prepare_time_html['title'][start_prepare_time:end_prepare_time]

                    #If recipe is more than a day:
                    if 'Day' in prepare_time_not_converted:
                        time_analyse = prepare_time_not_converted.split('Day')
                        time_hours = time_analyse[1].split('H')
                        #To convert into minutes (add the days and hours to the minutes)
                        if len(time_hours) == 1:
                            prepare_time = int(time_analyse[0])*60*24 + int(time_hours[0].replace('M',''))
                        if len(time_hours) == 2:
                            time_minute = time_hours[1].replace('M','');
                            prepare_time = int(time_analyse[0])*60*24 + int(time_hours[0])*60 + (int(time_minute) if time_minute else 0) 
                    #If the recipe is only in hours and minutes        
                    else:
                        time_analyse = prepare_time_not_converted.split('H')
                        #To convert hours into minutes
                        if len(time_analyse) == 1:
                            prepare_time = int(time_analyse[0].replace('M',''))
                        if len(time_analyse) == 2:
                            time_minute = time_analyse[1].replace('M','');
                            prepare_time = int(time_analyse[0])*60 + (int(time_minute) if time_minute else 0)
                    
                
                #Find the ingredient list:
                list_ingred = []
                ingredients = soup.find('div', class_='ingredients')
                all_ingredients = ingredients.find_all('li',class_="plaincharacterwrap ingredient")
                for ingredient in all_ingredients:
                    ingredient_i = ingredient.text.replace('\n','').lower()
                    #print('Original:', ingredient_i)
                    ingredient_i = ingredient_i.split(',')[0]
                    ingredient_i = [word for word in ingredient_i.split(' ') if word not in list_ingredient_to_remove] #Clean string to only have the ingredient
                    ingredient_i = ' '.join(x for x in ingredient_i if x.isalpha()) + ' '
                    ingredient_i = ingredient_i.replace(' or ', '//').replace('and ','//').replace(' with ','//').split('//') #if options, add both in the list 
                    #print('After removing:              ', ingredient_i,"\n")
                    
                    for ingredient_in_list in ingredient_i:
                        ingredient_in_list_strip = ingredient_in_list.strip()
                        if ingredient_in_list_strip == '':
                            continue
                        if ingredient_in_list_strip[len(ingredient_in_list_strip)-1] == 's' and ingredient_in_list_strip[0:len(ingredient_in_list_strip)-1] in list_unique_ingredients:
                            ingredient_in_list_strip = ingredient_in_list_strip[0:len(ingredient_in_list_strip)-1] #Remove the plural form (s) of the ingredient
                        
                        list_ingred.append(ingredient_in_list_strip) #Add the element to the ingredient list
                        if ingredient_in_list_strip not in list_unique_ingredients: 
                            list_unique_ingredients.append(ingredient_in_list_strip)
                            
                    
                #print(list_ingred)    
                
                recipe_data = recipe_data.append({'Website': website_list_used[0], 'Recipe': recipe_name, 'Ranking': rating, 'Reviews': review,\
                                                  'Ingredients': list_ingred}, ignore_index=True)
                
                
                #print('\n')
print(list_unique_ingredients)

We don't care about this page:  recipePagesT/00a1d99a6383cdeac3d6c253f8e7cff1.html
We don't care about this page:  recipePagesT/00a7450a98369de9097e042172e59fa4.html
We can't read the page: recipePagesT/00ad09075dfdaf485226fa532875e901.html
We don't care about this page:  recipePagesT/00bd78a7daa32aecada051639a3e029e.html
We don't care about this page:  recipePagesT/00d94e3c80215d3757282b09f211474e.html
We don't care about this page:  recipePagesT/00ef721d1c594ce09abea90254d2fcb9.html
We don't care about this page:  recipePagesT/00f954a1aa238510f977970cf7044865.html
We can't read the page: recipePagesT/00fd81c318c62be747ca4fb02a8da144.html
We don't care about this page:  recipePagesT/0a098768f9cd184563b28abb585e0c8d.html
We don't care about this page:  recipePagesT/0a2bff27d05456d28b90620e67f1ac69.html
We don't care about this page:  recipePagesT/0a391f5133008eaab71531f1fbfffa6d.html
We can't read the page: recipePagesT/0a3c08545503257db1069708918b94d6.html
We can't read the page: reci

In [11]:
len(list_unique_ingredients)


420

In [12]:
list_all_ingredients = []
for data in recipe_data['Ingredients']:
    #print(data)    
    #ingredients_strip = data.strip()
    for i in range(len(data)):
        list_all_ingredients.append(data[i])
    
    
print(len(list_all_ingredients))
list_unique_ingredients = set(list_all_ingredients)
print(len(list_unique_ingredients))
set(list_unique_ingredients)





1755
420


{'acorn squash',
 'active yeast',
 'adobo',
 'allspice',
 'almond',
 'almonds',
 'amaranth flour',
 'amaretto liqueur',
 'angel food cake',
 'anise',
 'apple',
 'apple cider vinegar',
 'apple juice',
 'arrowroot flour',
 'artichoke hearts',
 'asparagus',
 'avocado',
 'avocados',
 'baby greens',
 'bacon',
 'baking flour',
 'baking potato',
 'baking powder',
 'baking soda',
 'balsamic vinegar',
 'banana',
 'barbeque sauce',
 'basil',
 'basil sauce',
 'bay',
 'beans',
 'beef',
 'beef bouillon',
 'beef broth',
 'biscuit baking',
 'black beans',
 'black drain',
 'black pepper',
 'black peppercorns',
 'blue cheese',
 'bok choy',
 'bread',
 'bread flour',
 'breaded chicken tenders',
 'broccoli',
 'brownie',
 'brussels sprouts',
 'bulb garlic',
 'bulk pork breakfast sausage',
 'bunches cilantro',
 'butter',
 'buttermilk',
 'cabbage',
 'cajun',
 'cake flour',
 'cannellini beans',
 'canola oil',
 'capers',
 'carbonated beverage',
 'carrot',
 'carrots',
 'cauliflower',
 'cayenne pepper',
 'celery

In [352]:
"sprigs","sage","roma","romano","pasta","reduced","sodium","refried","refrigerated","portobello","nuggets","tater","pinto","pudding","pitted","pinches","relish","penne"




recipe_data

,Website,Recipe,Ranking,Reviews,Ingredients
0,allrecipes.com,Chicken Breast Cutlets with Artichokes and Capers,4.4,80,"[wheat, white flour, salt, white pepper, black..."
1,allrecipes.com,Best Ever Popcorn Balls,4.4,322,"[corn syrup, margarine, cold water, sugar, mar..."
2,allrecipes.com,Orange Cream Cheese Frosting,4.6,39,"[cream cheese, butter, orange zest, orange jui..."
3,allrecipes.com,Perfect Baked Potato,4.7,410,"[baking potato, olive oil, salt, butter, black..."
4,allrecipes.com,Pumpkin Oatmeal,3.9,88,"[rolled oats, milk, pumpkin puree, pumpkin pie..."
...,...,...,...,...,...
192,allrecipes.com,Homemade Wonton Soup,4.8,126,"[green onion, mushrooms, pork, sesame oil, soy..."
193,allrecipes.com,Rich Penne Pasta,3.8,32,"[penne pasta, link sausage, onion, cloves garl..."
194,allrecipes.com,Breaded Pork Chops,4.3,39,"[pork, flour, salt, paprika, pepper, egg, worc..."
195,allrecipes.com,Cocktail Meatballs I,4.6,226,"[beef, bread, onion, milk, egg, salt, worceste..."
